In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# get environment variable: OPENAI_API_KEY
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import tiktoken
import openai
import time
import numpy as np

In [4]:
# find tokens for 'positive' and 'negative'
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
enc.encode("positive"), enc.encode("negative") 

([31587], [43324])

In [5]:
# download dataset from hf datasets https://github.com/huggingface/datasets
# !pip install datasets 
from datasets import load_dataset

dataset = load_dataset("sst2")

Found cached dataset sst2 (/home/survival/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
val_data = dataset["validation"]

In [7]:
# sentimentid2sentimentlabel = {"0": "negative", "1": "positive"}
sents, labels = [], []
for idx in range(len(val_data)):
    sents.append(val_data[idx]["sentence"])
    label = val_data[idx]["label"]
    if label == 0:
        labels.append('negative')
    else:
        labels.append('positive')

In [8]:
def my_classifier(text: str) -> str:
    return openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {
            'role': 'system',
            'content': 'You are a helpful sentiment analyzer assistant of movie reviews'
        },
        {
        'role':'user',
        'content':f'Given the following text, what is the sentiment conveyed? Is it positive or negative?: {text}'
    }],
    logit_bias={
        '31587': 100,    # 31587 is the token for `positive`
        '43324': 100     # 43324 is the token for `negative`
    },
    max_tokens=1,
    temperature=0
).choices[0].message.content

In [9]:
preds = []
for i, sent in enumerate(sents):
    try:
        preds.append(my_classifier(sent))
    except:
        preds.append(np.nan)
        time.sleep(3)

In [10]:
import pandas as pd

missing = [i for i, pred in enumerate(preds) if pd.isna(pred)]
missing

[90, 279, 301, 742]

In [11]:
missing_preds = []
for idx in missing:
    try:
        missing_preds.append(my_classifier(sents[idx]))
    except:
        missing_preds.append(np.nan)
        time.sleep(3)

In [12]:
for i, idx in enumerate(missing):
    preds[idx] = missing_preds[i]

In [13]:
np.mean([labels[i] == preds[i] for i in range(len(preds))])

0.9587155963302753

In [14]:
results = pd.DataFrame()
results["sentence"] = sents
results["label"] = labels
results["predictions"] = preds

In [15]:
results.to_csv("./outputs/SST2_dataset_results_gpt-3.5-turbo.csv", index=False)